In [ ]:
import shutil
import os
import numpy as np
import matplotlib.pylab as plt

import sys
sys.path.append(os.path.abspath("../../framework"))
sys.path.append(os.path.abspath("../../concrete"))
sys.path.append(os.path.abspath("../../snippet"))

In [ ]:
from ConcAgent import ConcAgent
from ConcEnvironment import ConcEnvironment
from ConcEnvrionmentFactory import ConcEnvironmentFactory
from ConcAgentFactory import ConcAgentFactory
from ConcAgentMemento import ConcAgentMemento
from ConcBuildOrder import ConcBuildOrder
from ConcStore import ConcStore
from ConcObservation import ConcObservation
from framework import ClosedLoopSimulator
from framework import ObservationSequence 
from framework import TrainId, StoreField
from workUtils import *

## S100. Misc

### SS130. define static variables

In [ ]:
concStore = ConcStore()
agentFactory = ConcAgentFactory()
environmentFactory = ConcEnvironmentFactory()

### SS120. load trainlog into a database

In [ ]:
trainLogFolderPath = "./trainLog"
cur = loadTrainLog(trainLogFolderPath)

# Table TrainLog(
#     buildOrderId Text, /* accord to a build order */
#     trainId   Text, /* accord to a trained agent along with a simulation time*/
#     timeSimulation Integer,
#     timestamp timestamp
# )

## S200. Prepare a simulation

### SS210. choose a build order => SKIPPED

select a target build order id from the following list ...

In [ ]:
cur.execute('''
Select
    Timestamp
    , buildOrderId
    , TrainId
    , count(*)
    From TrainLog
    Group By buildOrderId
    Order by Timestamp desc
    ''')

print("Timestamp, buildOrderId, trainId, count")
for row in cur.fetchall():
    print(row)

Or, select a target build order id, randomly ...

In [ ]:
cur.execute('''
Select
    distinct buildOrderId
    From TrainLog
    ''')

buildOrderIds = [elm for elm, in cur.fetchall()] # contains all the build order ids
print(buildOrderIds)
buildOrderIdGiven = np.random.choice(buildOrderIds)

Or, select a target build order id as the latest trained one...

In [ ]:
cur.execute('''
Select
    buildOrderId
    From TrainLog
    Order By timestamp desc
    Limit 1
    ''')

buildOrderIdGiven = cur.fetchone()[0]

### SS220. choose the iteration(timesimulation) over the build with the given build order:  => SKIPPED

select a timeSimulation(an iteration number of the specified build process) from timeSimulations ...

In [ ]:
cur.execute('''
Select
    Distinct TimeSimulation
    From TrainLog
    Order By timeSimulation asc
    ''')
timeSimulations = [elm for elm, in cur.fetchall()] # (1, 100, 200, ...)
print(timeSimulations)

In [ ]:
timeSimulationGiven = int(np.random.choice(timeSimulations))

### SS225. define the iterator to generate the pair of buildOrderId and timeSimulation:

In [ ]:
def generateBuildOrderIdAndTimeSimulation():
    cur.execute('''
Select
    distinct buildOrderId
    From TrainLog
    ''')

    buildOrderIds = [elm for elm, in cur.fetchall()] # contains all the build order ids
    
    for buildOrderIdGiven in buildOrderIds:
        
        cur.execute('''
Select
    Distinct TimeSimulation
    From TrainLog
    Where buildOrderId = ?
    Order By timeSimulation asc
    ''', (buildOrderIdGiven,))
        
        timeSimulations = [elm for elm, in cur.fetchall()] # (1, 100, 200, ...)
        
        timeSimulationsToExport = [timeSimulations[i] for i in np.linspace(1, len(timeSimulations)-1, 4).astype(np.int)]
        
        for timeSimulationGiven in timeSimulationsToExport:            
            yield buildOrderIdGiven, timeSimulationGiven

## SS230. restore the agent and the environment

In [ ]:
def restoreAgentAndEnvironment(buildOrderIdGiven, timeSimulationGiven):
    cur.execute('''
        Select 
            trainId
            From TrainLog
            Where buildOrderId = ?
            And timeSimulation = ?
            ''', (buildOrderIdGiven, timeSimulationGiven))

    trainIdStr, = cur.fetchone()

    # storeField contains: agentMement, buildOrder
    storeField = concStore.load(TrainId(trainIdStr))

    # restore agent
    agent = agentFactory.create(storeField.getBuildOrder())
    agent.loadFromMemento(storeField.getAgentMemento())

    # restrore environment
    environment = environmentFactory.create(storeField.getBuildOrder())
    
    # restore nSeq
    nSeq = storeField.buildOrder.getnSeq()
    
    # restore weightOnError
    weightOnError = storeField.buildOrder.weightOnError
    
    return environment, agent, nSeq, weightOnError

In [ ]:
# environment, agent, nSeq = restoreAgentAndEnvironment(buildOrderIdGiven, timeSimulationGiven)

## S300. Run a simulation of the given pair of (environment, agent)

In [ ]:
# nSimu = 2**7

### SS310: run a simulation

In [ ]:
def runSimulation(environment, agent, nSeq, nSimu):
    
    Y = [] # observations
    U = [] # actions

    closedLoopSimulator = ClosedLoopSimulator(environment, agent, nSeq)
    closedLoopSimulator.init()
    y = closedLoopSimulator.observationSequenceLast[-1].getValue() # (1, nPv)
    Y.append(y)
    for _ in range(nSimu):
        closedLoopSimulator.update()
        y = closedLoopSimulator.observationSequenceLast[-1].getValue() # (1, nPv)
        u = closedLoopSimulator.actionLast.getActionOnEnvironment() # (1, nMv)
        Y.append(y)
        U.append(u)

    Ynumpy = np.concatenate(Y, axis=0) # (nSimu+1, nPv)
    Unumpy = np.concatenate(U, axis=0) # (nSimu, nMv)

    Ty = np.arange(nSimu+1) # (0, 1, ..., 9, 10)
    Tu = np.arange(nSimu) + 0.5 # (0.5, 1.5, 2.5, ..., 9.5)
    
    return Ty, Ynumpy, Tu, Unumpy

In [ ]:
# Ty, Ynumpy, Tu, Unumpy = runSimulation(environment, agent, nSeq)

### SS320: export the trace of observations and actions to a figure => SKIPPED

In [ ]:
# fig = plt.figure()
# fig.clf()
# #
# ax = fig.add_subplot(2,1,1)
# ax.plot(Ty, Ynumpy)
# ax.set_title('Observation')
# #
# ax = fig.add_subplot(2,1,2)
# ax.plot(Tu, Unumpy)
# ax.set_title('Action')
# #
# plt.tight_layout()

### SS330: export the trace of observations and actions to a csv-formatted file

In [ ]:
nSimu = 2**7

fmt1 = "iterations = {0}"
fmt2 = "weightOnError = {0}"

data = []
for buildOrderIdGiven, timeSimulationGiven in generateBuildOrderIdAndTimeSimulation():

    environment, agent, nSeq, weightOnError = restoreAgentAndEnvironment(buildOrderIdGiven, timeSimulationGiven)
    
    Ty, Ynumpy, Tu, Unumpy = runSimulation(environment, agent, nSeq, nSimu)
    
    data += [(t, *val, "Pv", fmt1.format(timeSimulationGiven), fmt2.format(weightOnError)) for (t, val) in zip(Ty, Ynumpy)]
    data += [(t, *val, "Mv", fmt1.format(timeSimulationGiven), fmt2.format(weightOnError)) for (t, val) in zip(Tu, Unumpy)]

In [ ]:
rows = ["time,value,varType,epoch,weightOnError",]\
        + [",".join(map(lambda xx: str(xx), row)) for row in data]
with open("./tmp/data.csv", "w") as fp:
    fp.write("\n".join(rows))